# visualize surprisal development


In [1]:
from pathlib import Path
import pandas as pd

ROOT = Path("/Users/jliu/workspace/RAG/")
surprisal_path = ROOT / "results" / "surprisal"

In [8]:
def format_csv(file_path:Path)->pd.DataFrame:
    """Remove the target col from the csv file."""
    df = pd.read_csv(file_path)
    #df = df.drop('ablated_neurons', axis=1)
    # convert differnet steps into different rows
    df_h = df[df["step"]==0]
    df_h = df_h.drop('surprisal', axis=1)
    df_grouped = df.groupby("step")
    for step, df_group, in df_grouped:
        df_h[step] = df_group['surprisal'].to_list()
    return df_h

In [9]:
# load freq file
file_path = surprisal_path/"pythia-example.csv"
df_h = format_csv(file_path)

In [10]:
df_h

,Unnamed: 0,step,target_word,context_id,context,0
0,0,0,airplane,0,Not only does this freshen up the look and fee...,12.577808
1,1,0,airplane,1,Many logos have evolved to take the shape of,11.117854
2,2,0,airplane,2,Pilot showcases stunning photos inside cockpit...,10.857062
3,3,0,airplane,3,"He is a young man who, prior to his unwilling ...",12.032984
4,4,0,airplane,4,And in 2011 he was taken from an,11.833903
...,...,...,...,...,...,...
95,95,0,asleep,7,French investigators said it was possible the ...,11.038956
96,96,0,asleep,8,“The crash may have been due to the driver fal...,11.320396
97,97,0,asleep,9,I wake up a t 5:00 a.m. on the dot every morni...,10.346714
98,98,0,asleep,10,We got back to her house and got blankets and ...,10.662737
